## Core Fisici vs Thread Logici

- **Core Fisico**: Hardware reale dentro la CPU.
  * Unità di elaborazione fisica
  * Ha proprie ALU (Arithmetic Logic Unit), registri, cache L1/L2
  * Esegue istruzioni indipendentemente dagli altri core

  4 core = 4 unità di calcolo fisiche separate.

- **Thread Logici**: Astrazione software che sfrutta risorse del core.
  * "Flussi di esecuzione" che il core può gestire
  * Con Hyper-Threading (tecnologia Intel): 1 core fisico = 2 thread logici

- **Hyper-Threading**: Il core ha risorse duplicate (registri, program counter)
Quando thread A si blocca (es. attende memoria), il core switcha istantaneamente a thread B

- Core fisico: cucina con fornelli, forno, piano di lavoro (Risorse)
- Hyper-Threading: 1 cucina = 2 cuochi che condividono attrezzature

Quando cuoco A aspetta che l'acqua bolla, cuoco B usa il piano di lavoro. Entrambi lavorano, ma condividono le risorse fisiche.

---

# Job System

Il Job System ottimizza l’uso della CPU trasformando il multitasking pesante in una gestione agile di piccoli carichi di lavoro:

- **Worker Thread e Core Logici**: Invece di creare thread arbitrari, il sistema ne alloca uno per ogni core logico della CPU, sfruttando l'hardware al 100% senza sovraccaricarlo.
- **Dai Task alla Job Queue**: Piuttosto che assegnare un intero compito (task) a un thread specifico, i lavori vengono frammentati in piccoli Job racchiusi in una coda (queue).
- **Efficienza Operativa**: I Worker Thread "pescano" continuamente dalla coda. Questo elimina il costo di gestione dei singoli thread, riduce lo spreco di risorse e permette di eseguire una mole enorme di calcoli in parallelo.

### RIASSUMENDO
I Job System lavorano in un Worker Thread, ma essenziale per 1 core logico...ma la positività dei job system è che sono semplicemente piccoli e racchiusi in una coda "queque". Quindi piuttosto che chiamare "task" per thread, raggruppiamo tutte quelle task in una coda, in un worker thread...quelle task vengono pescate dal pool. 

---

### 1. Obiettivo e Vantaggi

Il **C# Job System** permette di scrivere codice **multithreaded** sicuro e performante in Unity.

* **Performance:** Aumenta il frame rate e, combinato con il **Burst Compiler**, riduce il consumo di batteria (codice macchina ottimizzato).
* **Integrazione:** Si integra nativamente con il sistema interno di Unity, condividendo gli stessi **Worker Threads** per evitare conflitti sulla CPU.

### 2. Architettura: Worker Threads vs Thread Classici

* **Problema dei Thread Classici:** Creare un thread per ogni piccola operazione causa overhead e **Context Switching** (la CPU perde tempo a salvare/caricare stati invece di lavorare), saturando il sistema.
* **Soluzione Job System:** Usa un numero fisso di **Worker Threads** (solitamente uno per core logico della CPU). I "Job" (piccoli compiti) vengono messi in una **Job Queue**. I Worker prelevano i job dalla coda ed eseguono il lavoro in sequenza, mantenendo la CPU sempre attiva senza sprechi.

### 3. Sicurezza e Race Conditions

Il sistema protegge dai **Race Conditions** (bug causati da thread che accedono agli stessi dati contemporaneamente con tempistiche imprevedibili).

* **Isolamento:** I Job lavorano su **copie** dei dati, non sui riferimenti originali (usando tipi *blittable*).
* **Dipendenze:** Il sistema gestisce l'ordine di esecuzione. Se il Job A prepara dati per il Job B, il sistema assicura che B parta solo dopo che A ha finito.

### 4. NativeContainer: Memoria Condivisa

Poiché i Job usano copie dei dati, per riportare i risultati al thread principale è necessario usare i **NativeContainer**.

* Sono wrapper C# per la memoria nativa (non gestita dal Garbage Collector standard).
* Esempi: `NativeArray`, `NativeList`, `NativeQueue`.
* **Sicurezza Integrata:** Includono controlli (es. `AtomicSafetyHandle`) per prevenire errori di lettura/scrittura simultanea. L'attributo `[ReadOnly]` permette a più job di leggere lo stesso dato contemporaneamente in sicurezza.

### 5. Gestione della Memoria (Allocators)

Quando crei un NativeContainer, devi specificare quanto deve durare la memoria:

1. **Allocator.Temp:** Velocissimo, dura **1 frame**. (Non usabile nei Job).
2. **Allocator.TempJob:** Veloce, dura fino a **4 frame**. (Standard per la maggior parte dei Job piccoli).
3. **Allocator.Persistent:** Più lento, dura per tutta la vita dell'app (simile a `malloc`).

---

[Creating jobs](https://docs.unity3d.com/2020.1/Documentation/Manual/JobSystemCreatingJobs.html)

<!-- ### Cos'è una Race Condition

Output di un'operazione dipende dal timing di un altro processo.

'''text
Main Thread: legge variabile X
Job Thread: scrive variabile X

Chi arriva prima? → Risultato imprevedibile
''' -->